In [2]:
import subprocess   
def count_sequences_in_fasta(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file if line.startswith('>'))
import os

# Get the current PATH
current_path = os.environ.get('PATH')

# Define the directories to add
meme_bin_dir = os.path.join(os.environ['HOME'], 'meme', 'bin')
meme_libexec_dir = os.path.join(os.environ['HOME'], 'meme', 'libexec', 'meme-5.5.5')

# Add the directories to the PATH
os.environ['PATH'] = meme_bin_dir + os.pathsep + meme_libexec_dir + os.pathsep + current_path
intergen_dir="/home/davide/Desktop/genomiChro/intergeniche_RefSeq"
motivo="/home/davide/Downloads/motivo16.meme"

import pandas as pd
df=pd.DataFrame(columns=["num_geni","num_motivi"])
df.index.name="Ceppo"
for file in os.listdir(intergen_dir):
    if file.endswith(".fasta"):
        name=file[file.find("_"):file.find("_GCF")]
        name=name.split("_")[2:]
        name="_".join(name)
        df.loc[name,"num_geni"]=count_sequences_in_fasta(os.path.join(intergen_dir,file))
        #create directory for each file in intergen_dir/fimo
        dir_name=os.path.join(intergen_dir,"fimo",file[:-6])
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)

        command = f"fimo --oc {dir_name} --verbosity 1 --bgfile --nrdb-- --thresh 1.0E-4 --norc {motivo} {os.path.join(intergen_dir,file)}"
        #os.system(command)

In [3]:
fimo_dir="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo"
import os
file="best_site.narrowPeak"

for dir in os.listdir(fimo_dir):
    name=dir[dir.find("_"):dir.find("_GCF")]
    name=name.split("_")[2:]
    name="_".join(name)
    for file in os.listdir(os.path.join(fimo_dir,dir)):
        if file.endswith(".narrowPeak"):
            with open(os.path.join(fimo_dir,dir,file),"r") as f:
                lines = f.readlines()
            gene=[line.split()[0] for line in lines]
            num_genes_with_motif=len(set(gene))
            df.loc[name,"num_motivi"]=f"{num_genes_with_motif}({round(num_genes_with_motif/df.loc[name,'num_geni']*100,2)}%)"

In [4]:
df

,num_geni,num_motivi
Ceppo,,
CCMEE_29,5778,171(2.96%)
CCNUC1,5987,201(3.36%)
TS-821,4374,178(4.07%)
SAG_2025,7385,271(3.67%)
CCALA_043,6569,214(3.26%)
CCALA_051,5661,198(3.5%)
[FACHB-1243],6077,212(3.49%)
SAG_39.79,7547,258(3.42%)
PCC_7203,5696,207(3.63%)


In [5]:
intergen_refseq="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo"
genbank_dir="/home/davide/Desktop/genomiChro/annotati_Refseq"
from Bio import SeqIO
locus_to_proteinid = {}
ceppo_motivi={}
def count_GOs(ceppo,genbank_dir, fimo_best):
    with open(fimo_best, "r") as f:
        content = f.readlines()
    names = [line.split()[0] for line in content]
    GOs = []
    GOs_background = []
    go_to_genes = {}
    locus_to_proteinid[ceppo] = {}
    ceppo_motivi[ceppo]=names
    i,j=0,0
    for file2 in os.listdir(genbank_dir):
        if ceppo in file2:
            genoma = SeqIO.parse(os.path.join(genbank_dir,file2), "genbank")
            for seq in genoma:
                for record in seq.features:
                    if "locus_tag" in record.qualifiers and "protein_id" in record.qualifiers:
                        locus_to_proteinid[ceppo][record.qualifiers["locus_tag"][0]] = record.qualifiers["protein_id"][0]
                    gene=record.qualifiers.get("gene", ["None"])[0]
                    if "locus_tag"in record.qualifiers and record.qualifiers["locus_tag"][0] in names:
                        i+=1
                        go_terms = record.qualifiers.get("GO_process", ["None"])
                        for go_term in go_terms:
                            if go_term != "None":
                                GOs.append(go_term)
                                if go_term in go_to_genes:
                                    go_to_genes[go_term].append(gene)
                                else:
                                    go_to_genes[go_term] = [gene]
                    else:
                        j+=1
                        GOs_background.append([*record.qualifiers.get("GO_process", ["None"])])
    GOs = [item for item in GOs if item != "None"]
    GOs_background = [item for sublist in GOs_background for item in sublist if item != "None"]
    return GOs, GOs_background
from collections import Counter
for dir in os.listdir(intergen_refseq):
    for file in os.listdir(os.path.join(intergen_refseq,dir)):
        if file.endswith(".narrowPeak"):
            subname=dir[dir.find("_"):dir.find("_GCF")]
            name=subname.split("_")[2:]
            name="_".join(name)
            fimo_best=os.path.join(intergen_refseq,dir,file)
            GOs, GOs_background = count_GOs(subname,genbank_dir, fimo_best)

            GOs_counter = Counter(GOs)
            GOs_background_counter = Counter(GOs_background)
            for go in GOs_counter:
                df.loc[name,go] = f"{GOs_counter[go]} ({round(GOs_counter[go]/(GOs_counter[go]+GOs_background_counter[go])*100,2)}%)" 
        df=df.copy()

In [6]:
df
#order columns by number of non Nan values
df = df[df.notna().sum().sort_values(ascending=False).index]
#first column is number of genes and second is number of motifs
df = df[["num_geni","num_motivi"]+[col for col in df.columns if col not in ["num_geni","num_motivi"]]]
df
#delete last row
df


,num_geni,num_motivi,GO:0006310 - DNA recombination [Evidence IEA],GO:0055085 - transmembrane transport [Evidence IEA],GO:0009432 - SOS response [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0006355 - regulation of DNA-templated transcription [Evidence IEA],GO:0006412 - translation [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0006282 - regulation of DNA repair [Evidence IEA],...,GO:0009240 - isopentenyl diphosphate biosynthetic process [Evidence IEA],GO:0006420 - arginyl-tRNA aminoacylation [Evidence IEA],GO:0006177 - GMP biosynthetic process [Evidence IEA],GO:0008360 - regulation of cell shape [Evidence IEA],GO:0006096 - glycolytic process [Evidence IEA],GO:0051068 - dihydrolipoamide metabolic process [Evidence IEA],GO:0006750 - glutathione biosynthetic process [Evidence IEA],GO:0006423 - cysteinyl-tRNA aminoacylation [Evidence IEA],GO:0051301 - cell division [Evidence IEA],GO:0042779 - tRNA 3'-trailer cleavage [Evidence IEA]
Ceppo,,,,,,,,,,,,,,,,,,,,,
CCMEE_29,5778,171(2.96%),5 (17.24%),1 (2.33%),1 (100.0%),1 (10.0%),2 (6.67%),2 (3.57%),2 (10.53%),1 (100.0%),...,NaN,NaN,NaN,NaN,1 (11.11%),1 (100.0%),1 (20.0%),1 (50.0%),1 (50.0%),NaN
CCNUC1,5987,201(3.36%),5 (15.15%),2 (2.33%),1 (50.0%),3 (20.0%),6 (5.77%),1 (1.82%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TS-821,4374,178(4.07%),2 (10.0%),5 (6.94%),1 (50.0%),1 (6.67%),3 (4.29%),1 (1.79%),3 (15.79%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAG_2025,7385,271(3.67%),3 (5.08%),3 (3.12%),1 (33.33%),4 (30.77%),9 (6.77%),3 (5.36%),3 (14.29%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_043,6569,214(3.26%),4 (6.78%),2 (1.94%),1 (33.33%),1 (7.14%),7 (5.88%),1 (1.89%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_051,5661,198(3.5%),4 (12.12%),3 (3.49%),1 (33.33%),1 (6.67%),4 (3.96%),2 (3.57%),2 (9.09%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[FACHB-1243],6077,212(3.49%),2 (5.26%),4 (4.6%),1 (25.0%),2 (13.33%),4 (3.67%),1 (1.75%),3 (13.64%),1 (100.0%),...,1 (100.0%),1 (50.0%),1 (100.0%),1 (25.0%),NaN,NaN,NaN,NaN,NaN,NaN
SAG_39.79,7547,258(3.42%),8 (10.96%),2 (2.11%),1 (33.33%),2 (13.33%),8 (6.2%),1 (1.79%),3 (13.04%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCC_7203,5696,207(3.63%),2 (7.69%),1 (1.27%),1 (50.0%),2 (13.33%),6 (6.19%),2 (3.57%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 (100.0%)


In [7]:
df.to_csv("/home/davide/Desktop/tabella1.csv")

In [8]:
locustopid = {k: v for d in locus_to_proteinid.values() for k, v in d.items()}
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimoall/joined/best_site.narrowPeak"
names=[locustopid.get(line.split()[0],'') for line in open(file).readlines()]
names=[name for name in names if name!='']

In [81]:
import os
from bs4 import BeautifulSoup
from pprint import pprint
file=("/home/davide/Downloads/KEGGall.html")
soup=BeautifulSoup(open(file), 'html.parser')
li_tags = soup.find_all("li")
#delete first elements unitl one has Metabolic patwhays in text
for i, li in enumerate(li_tags):
    if "Metabolic pathways" in str(li):
        break
li_tags=li_tags[i:]

pathways = {}
#now iterate over the li tags and find the dd tags inside them
for li in li_tags:
    dd_tags = li.find_all("dd")
    pathway=str(li.contents[1])[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")

            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=['']:
                diz[key].extend(pids)
    pathways[pathway]=diz

for pathway in pathways:
    diz=pathways[pathway]
    for key in diz:
        diz[key]=len(set(diz[key]))

pprint(pathways)

{'2-Oxocarboxylic acid metabolism': {'CCALA_043': 33,
                                     'CCALA_051': 33,
                                     'CCMEE_29': 36,
                                     'CCNUC1': 33,
                                     'PCC_7203': 33,
                                     'SAG_2025': 35,
                                     'SAG_39.79': 39,
                                     'TS-821': 30,
                                     '[FACHB-1243]': 31},
 'ABC transporters': {'CCALA_043': 141,
                      'CCALA_051': 128,
                      'CCMEE_29': 110,
                      'CCNUC1': 125,
                      'PCC_7203': 123,
                      'SAG_2025': 137,
                      'SAG_39.79': 149,
                      'TS-821': 115,
                      '[FACHB-1243]': 130},
 'AMPK signaling pathway': {'CCALA_043': 5,
                            'CCALA_051': 3,
                            'CCMEE_29': 3,
                            'CCNU

In [82]:
from copy import deepcopy
pathways_motifs={}
for li in li_tags:
    dd_tags = li.find_all("dd")
    pathway=str(li.contents[1])[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(", ")
            pids=[pid for pid in pids if pid in names]
            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=[''] and len(pids)!=0:
                diz[key].extend(pids)
    pathways_motifs[pathway]=diz
pathways_genes=deepcopy(pathways_motifs)
#now for every pathway a counter of the proteins found in every ceppo
for pathway in pathways_motifs:
    diz=pathways_motifs[pathway]
    for key in diz:
        diz[key]=len(set(diz[key]))

In [83]:
pathways_genes
somma=0
for k,v in pathways_genes.items():
    for k2,v2 in v.items():
        somma+=len(v2)
somma


1205

In [84]:
df_pathway_genes=pd.DataFrame(pathways_genes.copy())

In [12]:
df_ko=pd.DataFrame(pathways_motifs)
df_ko
#order columns for sum of values
df_ko = df_ko[df_ko.sum().sort_values(ascending=False).index]
df_ko

,Metabolic pathways,Biosynthesis of secondary metabolites,Two-component system,Biosynthesis of cofactors,Mismatch repair,Homologous recombination,Microbial metabolism in diverse environments,Oxidative phosphorylation,DNA replication,Biosynthesis of amino acids,...,NF-kappa B signaling pathway,TNF signaling pathway,HIF-1 signaling pathway,Calcium signaling pathway,Phosphatidylinositol signaling system,Phospholipase D signaling pathway,PI3K-Akt signaling pathway,AMPK signaling pathway,Neuroactive ligand-receptor interaction,Steroid degradation
SAG_2025,23,12,7,6,7,5,5,3,4,4,...,0,0,0,0,0,0,0,0,0,0
PCC_7203,27,12,6,5,4,3,3,4,3,2,...,0,0,0,0,0,0,0,0,0,0
CCNUC1,27,14,5,6,5,5,3,3,4,2,...,0,0,0,0,0,0,0,0,0,0
CCMEE_29,17,6,3,4,3,4,4,0,2,2,...,0,0,0,0,0,0,0,0,0,0
TS-821,26,13,3,8,4,4,9,3,3,5,...,0,0,0,0,0,0,0,0,0,0
SAG_39.79,27,12,7,7,4,4,4,4,3,2,...,0,0,0,0,0,0,0,0,0,0
CCALA_043,25,11,8,6,4,5,2,4,3,2,...,0,0,0,0,0,0,0,0,0,0
CCALA_051,24,11,8,7,4,3,2,4,2,2,...,0,0,0,0,0,0,0,0,0,0
[FACHB-1243],25,13,7,5,5,3,1,3,3,2,...,0,0,0,0,0,0,0,0,0,0


In [13]:

#delete columns with only 0
df = df.loc[:, (df != 0).any(axis=0)]
df

,num_geni,num_motivi,GO:0006310 - DNA recombination [Evidence IEA],GO:0055085 - transmembrane transport [Evidence IEA],GO:0009432 - SOS response [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0006355 - regulation of DNA-templated transcription [Evidence IEA],GO:0006412 - translation [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0006282 - regulation of DNA repair [Evidence IEA],...,GO:0009240 - isopentenyl diphosphate biosynthetic process [Evidence IEA],GO:0006420 - arginyl-tRNA aminoacylation [Evidence IEA],GO:0006177 - GMP biosynthetic process [Evidence IEA],GO:0008360 - regulation of cell shape [Evidence IEA],GO:0006096 - glycolytic process [Evidence IEA],GO:0051068 - dihydrolipoamide metabolic process [Evidence IEA],GO:0006750 - glutathione biosynthetic process [Evidence IEA],GO:0006423 - cysteinyl-tRNA aminoacylation [Evidence IEA],GO:0051301 - cell division [Evidence IEA],GO:0042779 - tRNA 3'-trailer cleavage [Evidence IEA]
Ceppo,,,,,,,,,,,,,,,,,,,,,
CCMEE_29,5778,171(2.96%),5 (17.24%),1 (2.33%),1 (100.0%),1 (10.0%),2 (6.67%),2 (3.57%),2 (10.53%),1 (100.0%),...,NaN,NaN,NaN,NaN,1 (11.11%),1 (100.0%),1 (20.0%),1 (50.0%),1 (50.0%),NaN
CCNUC1,5987,201(3.36%),5 (15.15%),2 (2.33%),1 (50.0%),3 (20.0%),6 (5.77%),1 (1.82%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TS-821,4374,178(4.07%),2 (10.0%),5 (6.94%),1 (50.0%),1 (6.67%),3 (4.29%),1 (1.79%),3 (15.79%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAG_2025,7385,271(3.67%),3 (5.08%),3 (3.12%),1 (33.33%),4 (30.77%),9 (6.77%),3 (5.36%),3 (14.29%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_043,6569,214(3.26%),4 (6.78%),2 (1.94%),1 (33.33%),1 (7.14%),7 (5.88%),1 (1.89%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCALA_051,5661,198(3.5%),4 (12.12%),3 (3.49%),1 (33.33%),1 (6.67%),4 (3.96%),2 (3.57%),2 (9.09%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[FACHB-1243],6077,212(3.49%),2 (5.26%),4 (4.6%),1 (25.0%),2 (13.33%),4 (3.67%),1 (1.75%),3 (13.64%),1 (100.0%),...,1 (100.0%),1 (50.0%),1 (100.0%),1 (25.0%),NaN,NaN,NaN,NaN,NaN,NaN
SAG_39.79,7547,258(3.42%),8 (10.96%),2 (2.11%),1 (33.33%),2 (13.33%),8 (6.2%),1 (1.79%),3 (13.04%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PCC_7203,5696,207(3.63%),2 (7.69%),1 (1.27%),1 (50.0%),2 (13.33%),6 (6.19%),2 (3.57%),3 (13.64%),1 (100.0%),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 (100.0%)


In [14]:
file=("/home/davide/Downloads/KEGGBrite.html")
soup=BeautifulSoup(open(file), 'html.parser')
li_tags = soup.find_all("li")
for i, li in enumerate(li_tags):
    if "Enzymes" in str(li):
        break
li_tags = li_tags[i:]

brites = {}
#now iterate over the li tags and find the dd tags inside them
for li in li_tags:
    dd_tags = li.find_all("dd")
    brite=li.contents[0].get_text()+" - "+li.contents[1].get_text()[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(",")

            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=['']:
                diz[key].extend(pids)
    brites[brite]=diz

for brite in brites:
    diz=brites[brite]
    for key in diz:
        diz[key]=len(set(diz[key]))

pprint(brites)

{'ko00194 - Photosynthesis proteins': {'CCALA_043': 114,
                                       'CCALA_051': 110,
                                       'CCMEE_29': 86,
                                       'CCNUC1': 119,
                                       'PCC_7203': 116,
                                       'SAG_2025': 115,
                                       'SAG_39.79': 113,
                                       'TS-821': 93,
                                       '[FACHB-1243]': 110},
 'ko00199 - Cytochrome P450': {'CCALA_043': 1,
                               'CCALA_051': 1,
                               'CCMEE_29': 1,
                               'CCNUC1': 1,
                               'PCC_7203': 1,
                               'SAG_2025': 2,
                               'SAG_39.79': 1,
                               'TS-821': 1,
                               '[FACHB-1243]': 1},
 'ko00536 - Glycosaminoglycan binding proteins': {'CCALA_043': 2,
          

In [36]:
from copy import deepcopy
brites_motifs={}
for li in li_tags:
    dd_tags = li.find_all("dd")
    brite=li.contents[0].get_text()+" - "+li.contents[1].get_text()[:-1].strip()
    diz = {}
    for dd in dd_tags:
        contents = dd.contents
        contents=[c for c in contents if c.name != 'br']
        contents=[str(content).replace('\xa0', '') for content in contents]
        contents=[str(c) for c in contents]
        for i, e in enumerate(contents):
            e=e.split(":")
            pids=e[1].split(",")
            if "WP_250121656.1" in pids:
                print(brite)
            pids=[pid for pid in pids if pid in names]
            key=e[0]
            if key not in diz:
                diz[key]=pids
            elif pids!=[''] and len(pids)!=0:
                diz[key].extend(pids)
    brites_motifs[brite]=diz
    
brites_genes=deepcopy(brites_motifs)
for brite in brites_motifs:
    diz=brites_motifs[brite]
    for key in diz:
        diz[key]=len(set(diz[key]))


ko01000 - Enzymes
ko03400 - DNA repair and recombination proteins


In [37]:
#create dataframe with brites_motifs
df_brite=pd.DataFrame(brites_motifs)
df_brite
#delete columns with only 0
df_brite = df_brite.loc[:, (df_brite != 0).any(axis=0)]
df_brite

,ko01000 - Enzymes,ko01001 - Protein kinases,ko01002 - Peptidases and inhibitors,ko01003 - Glycosyltransferases,ko01011 - Peptidoglycan biosynthesis and degradation proteins,ko01004 - Lipid biosynthesis proteins,ko01008 - Polyketide biosynthesis proteins,ko01006 - Prenyltransferases,ko01007 - Amino acid related enzymes,ko00194 - Photosynthesis proteins,...,ko03029 - Mitochondrial biogenesis,ko02000 - Transporters,ko02044 - Secretion system,ko02022 - Two-component system,ko02035 - Bacterial motility proteins,ko04812 - Cytoskeleton proteins,ko04147 - Exosome,ko02048 - Prokaryotic defense system,ko01504 - Antimicrobial resistance genes,ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins
SAG_2025,43,4,3,0,1,0,1,2,1,2,...,1,6,1,5,3,1,1,2,0,1
PCC_7203,47,2,3,0,3,0,0,2,1,4,...,1,13,1,6,3,0,2,1,0,0
CCNUC1,48,1,4,0,3,0,0,2,1,4,...,1,12,1,5,3,0,2,1,0,0
CCMEE_29,31,0,4,0,1,1,0,0,2,2,...,2,4,0,1,0,0,1,2,0,0
TS-821,48,0,6,2,0,1,1,0,2,2,...,2,10,2,1,0,0,3,2,2,0
SAG_39.79,51,2,3,0,3,0,0,2,1,3,...,1,15,1,6,3,1,3,1,0,1
CCALA_043,46,2,4,0,3,0,0,2,1,3,...,1,14,1,6,3,1,3,1,0,1
CCALA_051,43,3,3,0,2,0,0,2,1,2,...,1,8,1,5,2,0,3,1,0,1
[FACHB-1243],41,2,4,1,1,0,1,2,1,2,...,2,9,1,6,3,0,2,1,0,0


In [38]:
#order columns for sum of values
df_brite = df_brite[df_brite.sum().sort_values(ascending=False).index]
#join df with df_brite
df=df.join(df_brite)
df

,p-value,enrichment,ko01000 - Enzymes,ko03400 - DNA repair and recombination proteins,ko02000 - Transporters,ko03032 - DNA replication proteins,ko02022 - Two-component system,ko01002 - Peptidases and inhibitors,ko03016 - Transfer RNA biogenesis,ko03036 - Chromosome and associated proteins,...,ko03110 - Chaperones and folding catalysts,ko03009 - Ribosome biogenesis,ko03019 - Messenger RNA biogenesis,ko03021 - Transcription machinery,ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins,ko01008 - Polyketide biosynthesis proteins,ko01003 - Glycosyltransferases,ko04812 - Cytoskeleton proteins,ko01004 - Lipid biosynthesis proteins,ko01504 - Antimicrobial resistance genes
ko term,,,,,,,,,,,,,,,,,,,,,
ko03400 - DNA repair and recombination proteins,9.643576e-39,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko03032 - DNA replication proteins,7.199285e-19,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko02022 - Two-component system,3.177998e-06,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko01006 - Prenyltransferases,1.289653e-05,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins,2.151163e-02,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko03016 - Transfer RNA biogenesis,2.912650e-02,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko01011 - Peptidoglycan biosynthesis and degradation proteins,5.493910e-02,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko03036 - Chromosome and associated proteins,1.046242e-01,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ko01001 - Protein kinases,1.697064e-01,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_brite_genes=pd.DataFrame(brites_genes.copy())
df_brite_genes


,ko01000 - Enzymes,ko01001 - Protein kinases,ko01009 - Protein phosphatases and associated proteins,ko01002 - Peptidases and inhibitors,ko01003 - Glycosyltransferases,ko01005 - Lipopolysaccharide biosynthesis proteins,ko01011 - Peptidoglycan biosynthesis and degradation proteins,ko01004 - Lipid biosynthesis proteins,ko01008 - Polyketide biosynthesis proteins,ko01006 - Prenyltransferases,...,ko02022 - Two-component system,ko02035 - Bacterial motility proteins,ko04812 - Cytoskeleton proteins,ko04147 - Exosome,ko02048 - Prokaryotic defense system,ko04030 - G protein-coupled receptors,ko01504 - Antimicrobial resistance genes,ko00536 - Glycosaminoglycan binding proteins,ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins,ko03200 - Viral proteins
SAG_2025,"[WP_317106986.1, WP_317105586.1, WP_317108507....","[WP_317110450.1, WP_317106342.1, WP_317105483....",[],"[WP_039714178.1, WP_317107225.1, WP_317106575.1]",[],[],[WP_317109188.1],[],[WP_317106964.1],"[WP_317104870.1, WP_317105824.1]",...,"[WP_317109238.1, WP_317110450.1, WP_317106342....","[WP_317109238.1, WP_317110450.1, WP_317106342.1]",[WP_317112577.1],[WP_317107894.1],"[WP_317105088.1, WP_317108679.1]",[],[],[],[WP_317107894.1],[]
PCC_7203,"[WP_015153605.1, WP_015153221.1, WP_015157227....","[WP_015153049.1, WP_015155069.1]",[],"[WP_015155906.1, WP_015153997.1, WP_015153147.1]",[],[],"[WP_015156634.1, WP_015155013.1, WP_015154962.1]",[],[],"[WP_219336057.1, WP_015153732.1]",...,"[WP_015156828.1, WP_015156687.1, WP_015153049....","[WP_015156687.1, WP_015153049.1, WP_015152127.1]",[],"[WP_015153221.1, WP_015157227.1]",[WP_015153637.1],[],[],[],[],[]
CCNUC1,"[WP_015153605.1, WP_106547394.1, WP_015153221....",[WP_015153049.1],[],"[WP_015155906.1, WP_250012819.1, WP_015153997....",[],[],"[WP_250013772.1, WP_250012305.1, WP_250012243.1]",[],[],"[WP_250015077.1, WP_106218904.1]",...,"[WP_106168935.1, WP_015156687.1, WP_015153049....","[WP_015156687.1, WP_015153049.1, WP_015152127.1]",[],"[WP_015153221.1, WP_015157227.1]",[WP_250016253.1],[],[],[],[],[]
CCMEE_29,"[WP_250123322.1, WP_250124543.1, WP_250126058....",[],[],"[WP_250125188.1, WP_250125712.1, WP_250121220....",[],[],[WP_250125712.1],[WP_250126058.1],[],[],...,[WP_250124410.1],[],[],[WP_250124543.1],"[WP_250125134.1, WP_256478979.1]",[],[],[],[],[]
TS-821,"[WP_104544953.1, WP_104548770.1, WP_104547256....",[],[],"[WP_104548671.1, WP_246841289.1, WP_210404370....","[WP_104547708.1, WP_104545712.1]",[],[],[WP_104546532.1],[WP_104546602.1],[],...,[WP_246841458.1],[],[],"[WP_104545694.1, WP_210404567.1, WP_104546532.1]","[WP_104546599.1, WP_104545196.1]",[],"[WP_104548237.1, WP_104545453.1]",[],[],[]
SAG_39.79,"[WP_322506111.1, WP_106168843.1, WP_106166858....","[WP_015153049.1, WP_106169547.1]",[],"[WP_015155906.1, WP_015153997.1, WP_015153147.1]",[],[],"[WP_199755431.1, WP_106170263.1, WP_106168445.1]",[],[],"[WP_241994356.1, WP_106171587.1]",...,"[WP_106168935.1, WP_015156687.1, WP_015153049....","[WP_015156687.1, WP_015153049.1, WP_015152127.1]",[WP_106171440.1],"[WP_106166858.1, WP_127022249.1, WP_199755926.1]",[WP_106166942.1],[],[],[],[WP_199755926.1],[]
CCALA_043,"[WP_106168843.1, WP_199755910.1, WP_258184061....","[WP_015153049.1, WP_106169547.1]",[],"[WP_015155906.1, WP_106165792.1, WP_015153997....",[],[],"[WP_106166534.1, WP_106170263.1, WP_106168445.1]",[],[],"[WP_241994356.1, WP_106171587.1]",...,"[WP_106168935.1, WP_015156687.1, WP_015153049....","[WP_015156687.1, WP_015153049.1, WP_015152127.1]",[WP_106171440.1],"[WP_199755910.1, WP_258184061.1, WP_199755926.1]",[WP_106166942.1],[],[],[],[WP_199755926.1],[]
CCALA_051,"[WP_106546890.1, WP_106546803.1, WP_106547394....","[WP_106545082.1, WP_233222557.1, WP_106544812.1]",[],"[WP_015155906.1, WP_015153997.1, WP_015153147.1]",[],[],"[WP_106545588.1, WP_199197503.1]",[],[],"[WP_233222594.1, WP_106548147.1]",...,"[WP_015156687.1, WP_106545082.1, WP_015153506....","[WP_015156687.1, WP_106545082.1]",[],"[WP_015157227.1, WP_041462369.1, WP_106544486.1]",[WP_1065

In [19]:
#save df to csv
df.to_csv("/home/davide/Desktop/tabellafinale.csv")


In [20]:
brites_joined= {k:sum(v.values()) for k,v in brites.items()}
brites_joined
brites_motifs_joined= {k:sum(v.values()) for k,v in brites_motifs.items()}

In [21]:
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimoall/joined/best_site.narrowPeak"
#gene = 
#names=[locustopid.get(line.split()[0],'') for line in open(file).readlines()]
input_file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa"
gene_dict = {}
with open(input_file, 'r') as file:  # replace 'filename.fasta' with your actual file path
    for line in file:
        if line.startswith('>'):
            parts = line.split()
            locus_tag = parts[0][1:]
            for i,part in enumerate(parts):
                if part=="gene" and parts[i+2][0].isalpha():
                    gene_name = parts[i+2]
                    break
                else:
                    gene_name = "None"
            if locus_tag in locustopid:
                gene_dict[locustopid[locus_tag]] = gene_name
   

gene_dict

{'WP_106170542.1': 'None',
 'WP_127025321.1': 'None',
 'WP_106170550.1': 'None',
 'WP_106170552.1': 'None',
 'WP_106170557.1': 'None',
 'WP_106170560.1': 'None',
 'WP_146138844.1': 'None',
 'WP_158631969.1': 'None',
 'WP_146138845.1': 'None',
 'WP_106170568.1': 'None',
 'WP_015153616.1': 'None',
 'WP_106170565.1': 'ychF',
 'WP_106170570.1': 'None',
 'WP_015153614.1': 'None',
 'WP_015153615.1': 'smpB',
 'WP_106166432.1': 'None',
 'WP_106166433.1': 'pyk',
 'WP_106166439.1': 'None',
 'WP_106166440.1': 'bchH',
 'WP_258183830.1': 'None',
 'WP_106166442.1': 'None',
 'WP_015155204.1': 'None',
 'WP_106166431.1': 'None',
 'WP_241994169.1': 'None',
 'WP_106166434.1': 'None',
 'WP_106166435.1': 'None',
 'WP_106166436.1': 'None',
 'WP_106166437.1': 'cax',
 'WP_106166438.1': 'cax',
 'WP_241994167.1': 'None',
 'WP_106166441.1': 'None',
 'WP_199755456.1': 'None',
 'WP_158631891.1': 'None',
 'WP_106170578.1': 'None',
 'WP_106170580.1': 'None',
 'WP_158631892.1': 'None',
 'WP_015157480.1': 'None',
 'WP

In [22]:
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa"
from scipy.stats import hypergeom
#count number of sequences in fasta file
def count_sequences_in_fasta(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file if line.startswith('>'))

M = count_sequences_in_fasta(file)
n = len(names)
p_values = {}
enrichments = {}
for ko in brites_motifs_joined:
    N = brites_joined[ko]
    x = brites_motifs_joined[ko]
    enrichments[ko] = (x/n)/(N/M)
    enrichments[ko] = 'positive' if enrichments[ko]>1 else 'negative'
    p_values[ko] = hypergeom.sf(x-1, M, N, n)
    #uso distribuzione ipergeometrica per calcolare p-values
df = pd.DataFrame(list(p_values.items()), columns=['ko term', 'p-value'])
df['enrichment'] = df['ko term'].map(enrichments)


df = df.sort_values(by='p-value', ascending=True)
pd.set_option('display.max_rows', df.shape[0]+1)

In [23]:
df

,ko term,p-value,enrichment
25,ko03400 - DNA repair and recombination proteins,9.643576e-39,positive
23,ko03032 - DNA replication proteins,7.199285e-19,positive
30,ko02022 - Two-component system,3.177998e-06,positive
9,ko01006 - Prenyltransferases,1.289653e-05,positive
38,ko00537 - Glycosylphosphatidylinositol (GPI)-a...,2.151163e-02,positive
18,ko03016 - Transfer RNA biogenesis,2.912650e-02,positive
6,ko01011 - Peptidoglycan biosynthesis and degra...,5.493910e-02,positive
24,ko03036 - Chromosome and associated proteins,1.046242e-01,positive
1,ko01001 - Protein kinases,1.697064e-01,positive
3,ko01002 - Peptidases and inhibitors,2.447340e-01,positive


In [24]:
for k,v in brites_genes.items():
    for k2,v2 in v.items():
        print(k2,v2)
        for i,el in enumerate(v2):
            if el in gene_dict and gene_dict[el]!="None":
                v2[i]=gene_dict[el]

SAG_2025 ['WP_317106986.1', 'WP_317105586.1', 'WP_317108507.1', 'WP_317108429.1', 'WP_317107420.1', 'WP_317104870.1', 'WP_317104666.1', 'WP_317110507.1', 'WP_317107894.1', 'WP_039714178.1', 'WP_317109188.1', 'WP_317108987.1', 'WP_317106010.1', 'WP_317105825.1', 'WP_317108098.1', 'WP_317105417.1', 'WP_317109189.1', 'WP_317106651.1', 'WP_317108552.1', 'WP_317109348.1', 'WP_015152415.1', 'WP_317105379.1', 'WP_317111456.1', 'WP_317105824.1', 'WP_317110450.1', 'WP_317108419.1', 'WP_317105752.1', 'WP_317106964.1', 'WP_317106575.1', 'WP_317104631.1', 'WP_015156433.1', 'WP_106218380.1', 'WP_317108570.1', 'WP_317107695.1', 'WP_317105088.1', 'WP_317108679.1', 'WP_317110020.1', 'WP_317105483.1', 'WP_317109341.1', 'WP_317112648.1', 'WP_015157307.1', 'WP_317110448.1', 'WP_317106042.1']
PCC_7203 ['WP_015153605.1', 'WP_015153221.1', 'WP_015157227.1', 'WP_015154222.1', 'WP_015156461.1', 'WP_015156998.1', 'WP_219336057.1', 'WP_015156874.1', 'WP_015154828.1', 'WP_015155906.1', 'WP_015156634.1', 'WP_0151

In [25]:
#set ko_term as index
df.set_index('ko term', inplace=True)
df

,p-value,enrichment
ko term,,
ko03400 - DNA repair and recombination proteins,9.643576e-39,positive
ko03032 - DNA replication proteins,7.199285e-19,positive
ko02022 - Two-component system,3.177998e-06,positive
ko01006 - Prenyltransferases,1.289653e-05,positive
ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins,2.151163e-02,positive
ko03016 - Transfer RNA biogenesis,2.912650e-02,positive
ko01011 - Peptidoglycan biosynthesis and degradation proteins,5.493910e-02,positive
ko03036 - Chromosome and associated proteins,1.046242e-01,positive
ko01001 - Protein kinases,1.697064e-01,positive


In [26]:
brites_string=deepcopy(brites_genes)
import json

brites_string= {outer_key: {inner_key: inner_value for inner_key, inner_value in outer_value.items() if inner_value != []} for outer_key, outer_value in brites_string.items()}
brites_string = {key: json.dumps(value) for key, value in brites_string.items() for key2,value2 in value.items()}
series_dict = {key: pd.Series(value) for key, value in brites_string.items()}
df_brites_string = pd.DataFrame(series_dict).T
df_brites_string
#rename con column to geni con motivo



,0
ko01000 - Enzymes,"{""SAG_2025"": [""WP_317106986.1"", ""WP_317105586...."
ko01001 - Protein kinases,"{""SAG_2025"": [""WP_317110450.1"", ""WP_317106342...."
ko01002 - Peptidases and inhibitors,"{""SAG_2025"": [""lexA"", ""sppA"", ""WP_317106575.1""..."
ko01003 - Glycosyltransferases,"{""TS-821"": [""WP_104547708.1"", ""WP_104545712.1""..."
ko01011 - Peptidoglycan biosynthesis and degradation proteins,"{""SAG_2025"": [""WP_317109188.1""], ""PCC_7203"": [..."
ko01004 - Lipid biosynthesis proteins,"{""CCMEE_29"": [""WP_250126058.1""], ""TS-821"": [""W..."
ko01008 - Polyketide biosynthesis proteins,"{""SAG_2025"": [""WP_317106964.1""], ""TS-821"": [""W..."
ko01006 - Prenyltransferases,"{""SAG_2025"": [""miaA"", ""WP_317105824.1""], ""PCC_..."
ko01007 - Amino acid related enzymes,"{""SAG_2025"": [""WP_317104666.1""], ""PCC_7203"": [..."
ko00194 - Photosynthesis proteins,"{""SAG_2025"": [""apcA"", ""WP_015153888.1""], ""PCC_..."


In [27]:
for k,v in pathways_genes.items():
    for k2,v2 in v.items():
        for i,el in enumerate(v2):
            if el in gene_dict and gene_dict[el]!="None":
                v2[i]=gene_dict[el]
pathways_string=deepcopy(pathways_genes)
pathways_string= {outer_key: {inner_key: inner_value for inner_key, inner_value in outer_value.items() if inner_value != []} for outer_key, outer_value in pathways_string.items()}
pathways_string = {key: json.dumps(value) for key, value in pathways_string.items() for key2,value2 in value.items()}
series_dict = {key: pd.Series(value) for key, value in pathways_string.items()}
df_pathways_string = pd.DataFrame(series_dict).T
df_pathways_string

,0
Metabolic pathways,"{""SAG_2025"": [""WP_317106986.1"", ""metE"", ""glgP""..."
Biosynthesis of secondary metabolites,"{""SAG_2025"": [""WP_317106986.1"", ""metE"", ""glgP""..."
Microbial metabolism in diverse environments,"{""SAG_2025"": [""WP_317106986.1"", ""WP_106218380...."
Carbon metabolism,"{""SAG_2025"": [""WP_106218380.1"", ""WP_317108570...."
2-Oxocarboxylic acid metabolism,"{""SAG_2025"": [""WP_317104666.1"", ""argB""], ""PCC_..."
...,...
Insulin resistance,"{""SAG_2025"": [""glgP""]}"
beta-Lactam resistance,"{""TS-821"": [""WP_246841293.1"", ""WP_104548237.1""]}"
Cationic antimicrobial peptide (CAMP) resistance,"{""SAG_2025"": [""WP_317109188.1""], ""PCC_7203"": [..."
Platinum drug resistance,"{""PCC_7203"": [""WP_015156874.1""], ""SAG_39.79"": ..."


In [28]:
pathways

{'Metabolic pathways': {'SAG_2025': 1037,
  'PCC_7203': 951,
  'CCNUC1': 981,
  'CCMEE_29': 884,
  'TS-821': 844,
  'SAG_39.79': 1035,
  'CCALA_043': 996,
  'CCALA_051': 957,
  '[FACHB-1243]': 919},
 'Biosynthesis of secondary metabolites': {'SAG_2025': 425,
  'PCC_7203': 397,
  'CCNUC1': 409,
  'CCMEE_29': 398,
  'TS-821': 349,
  'SAG_39.79': 428,
  'CCALA_043': 421,
  'CCALA_051': 393,
  '[FACHB-1243]': 370},
 'Microbial metabolism in diverse environments': {'SAG_2025': 228,
  'PCC_7203': 192,
  'CCNUC1': 204,
  'CCMEE_29': 183,
  'TS-821': 185,
  'SAG_39.79': 231,
  'CCALA_043': 222,
  'CCALA_051': 189,
  '[FACHB-1243]': 189},
 'Carbon metabolism': {'SAG_2025': 109,
  'PCC_7203': 98,
  'CCNUC1': 101,
  'CCMEE_29': 98,
  'TS-821': 93,
  'SAG_39.79': 108,
  'CCALA_043': 104,
  'CCALA_051': 97,
  '[FACHB-1243]': 94},
 '2-Oxocarboxylic acid metabolism': {'SAG_2025': 35,
  'PCC_7203': 33,
  'CCNUC1': 33,
  'CCMEE_29': 36,
  'TS-821': 30,
  'SAG_39.79': 39,
  'CCALA_043': 33,
  'CCALA_051

In [29]:
file = "/home/davide/Desktop/genomiChro/intergeniche_RefSeq/joined.fa"
from scipy.stats import hypergeom


#count number of sequences in fasta file
def count_sequences_in_fasta(file_path):
    with open(file_path, 'r') as file:
        return sum(1 for line in file if line.startswith('>'))

pathways_joined= {k:sum(v.values()) for k,v in pathways.items()}
pathways_motifs_joined= {k:sum(v.values()) for k,v in pathways_motifs.items()}
M = count_sequences_in_fasta(file)
n = len(names)
p_values = {}
enrichments = {}
for ko in pathways_motifs_joined:
    N = pathways_joined[ko]
    x = pathways_motifs_joined[ko]
    enrichments[ko] = (x / n) / (N / M)
    enrichments[ko] = 'positive' if enrichments[ko] > 1 else 'negative'
    p_values[ko] = hypergeom.sf(x - 1, M, N, n)
    #uso distribuzione ipergeometrica per calcolare p-values
df_pathway = pd.DataFrame(list(p_values.items()), columns=['pathway', 'p-value'])
df_pathway['enrichment'] = df_pathway['pathway'].map(enrichments)
df_pathway.sort_values(by='p-value', ascending=True,inplace=True)
pd.set_option('display.max_rows', df.shape[0] + 1)


In [30]:
df_pathway.set_index("pathway",inplace=True)
df_pathway

,p-value,enrichment
pathway,,
Mismatch repair,7.744156e-17,positive
Homologous recombination,2.961694e-12,positive
DNA replication,2.632612e-10,positive
Zeatin biosynthesis,5.183101e-06,positive
Nucleotide excision repair,5.835690e-06,positive
...,...,...
Endocytosis,1.000000e+00,negative
Lysosome,1.000000e+00,negative
Biosynthesis of type II polyketide products,1.000000e+00,negative


In [31]:

df_pathway=df_pathway.join(df_pathways_string)
df_pathway.rename(columns={0:"geni con motivo"},inplace=True)
#df_pathway["p-value"]=df_pathway["p-value"].round(2)
df_pathway



,p-value,enrichment,geni con motivo
pathway,,,
Mismatch repair,7.744156e-17,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Homologous recombination,2.961694e-12,positive,"{""SAG_2025"": [""WP_317108552.1"", ""WP_317109348...."
DNA replication,2.632612e-10,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Zeatin biosynthesis,5.183101e-06,positive,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
Nucleotide excision repair,5.835690e-06,positive,"{""SAG_2025"": [""ligA"", ""uvrB""], ""PCC_7203"": [""l..."
...,...,...,...
Endocytosis,1.000000e+00,negative,NaN
Lysosome,1.000000e+00,negative,NaN
Biosynthesis of type II polyketide products,1.000000e+00,negative,NaN


In [32]:
def round_pvalues(df):
    # Round p-values column while preserving exponential notation
    df['p-value'] = df['p-value'].apply(lambda x: '{:.2e}'.format(x))
round_pvalues(df_pathway)
df_pathway

,p-value,enrichment,geni con motivo
pathway,,,
Mismatch repair,7.74e-17,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Homologous recombination,2.96e-12,positive,"{""SAG_2025"": [""WP_317108552.1"", ""WP_317109348...."
DNA replication,2.63e-10,positive,"{""SAG_2025"": [""ligA"", ""WP_317108552.1"", ""WP_31..."
Zeatin biosynthesis,5.18e-06,positive,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
Nucleotide excision repair,5.84e-06,positive,"{""SAG_2025"": [""ligA"", ""uvrB""], ""PCC_7203"": [""l..."
...,...,...,...
Endocytosis,1.00e+00,negative,NaN
Lysosome,1.00e+00,negative,NaN
Biosynthesis of type II polyketide products,1.00e+00,negative,NaN


In [33]:
df_pathway.to_csv("/home/davide/Desktop/ArricchimentoKEGGpathways.csv")

In [25]:
#rename column 0
df_brites_string.columns=["geni con motivo"]
df_brites_string

,geni con motivo
ko01000 - Enzymes,"{""SAG_2025"": [""WP_317105586.1"", ""metE"", ""miaA""..."
ko01001 - Protein kinases,"{""SAG_2025"": [""WP_317106342.1"", ""WP_317105483...."
ko01002 - Peptidases and inhibitors,"{""SAG_2025"": [""lexA""], ""PCC_7203"": [""lexA""], ""..."
ko01011 - Peptidoglycan biosynthesis and degradation proteins,"{""PCC_7203"": [""WP_015154962.1""], ""CCNUC1"": [""W..."
ko01008 - Polyketide biosynthesis proteins,"{""SAG_2025"": [""WP_317106964.1""], ""TS-821"": [""W..."
ko01006 - Prenyltransferases,"{""SAG_2025"": [""miaA""], ""PCC_7203"": [""miaA""], ""..."
ko01007 - Amino acid related enzymes,"{""SAG_2025"": [""WP_317104666.1""], ""PCC_7203"": [..."
ko00194 - Photosynthesis proteins,"{""PCC_7203"": [""psb28""], ""CCNUC1"": [""psb28""], ""..."
ko03000 - Transcription factors,"{""SAG_2025"": [""WP_317110206.1""], ""PCC_7203"": [..."
ko03019 - Messenger RNA biogenesis,"{""SAG_2025"": [""ppk2""], ""PCC_7203"": [""ppk2""], ""..."


In [26]:
df=df.join(df_brites_string)

In [27]:
df
#drop genes column
df.drop(columns=["genes"],inplace=True)

KeyError: "['genes'] not found in axis"

In [28]:
df

,ko term,p-value,enrichment,geni con motivo
25,ko03400 - DNA repair and recombination proteins,5.827873e-17,positive,NaN
23,ko03032 - DNA replication proteins,2.205853e-06,positive,NaN
9,ko01006 - Prenyltransferases,3.181548e-02,positive,NaN
18,ko03016 - Transfer RNA biogenesis,5.215056e-02,positive,NaN
15,ko03019 - Messenger RNA biogenesis,6.469089e-02,positive,NaN
30,ko02022 - Two-component system,8.744878e-02,positive,NaN
8,ko01008 - Polyketide biosynthesis proteins,1.499579e-01,positive,NaN
13,ko03000 - Transcription factors,1.670386e-01,positive,NaN
3,ko01002 - Peptidases and inhibitors,3.758128e-01,positive,NaN
1,ko01001 - Protein kinases,4.318219e-01,positive,NaN


In [142]:
#save df to csv
df.to_csv("/home/davide/Desktop/arricchimentoKOBrite.csv")

In [88]:
df_pathway_genes.index

Index(['SAG_2025', 'PCC_7203', 'CCNUC1', 'CCMEE_29', 'TS-821', 'SAG_39.79',
       'CCALA_043', 'CCALA_051', '[FACHB-1243]'],
      dtype='object')

In [90]:
df_brite_genes.index==df_pathway_genes.index

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [129]:
from pathlib import Path
dir=Path("/home/davide/Desktop/genomiChro/intergeniche_RefSeq/fimo")
refseq=Path("/home/davide/Desktop/genomiChro/annotati_Refseq")
for file in dir.glob("**/fimo.tsv"):
    motivi=pd.read_csv(file,sep="\t",header=0,skipfooter=4,engine='python')
    motivi=motivi[["sequence_name",'matched_sequence']]
    motivi.columns=["locus","motivo"]
    motivi.set_index("locus",inplace=True)
    name=file.parts[-2]
    name=name[name.find("_"):name.find("_GCF")]
    name=name.split("_")[2:]
    name="_".join(name)
    brite = (df_brite_genes.loc[name]
         .explode()
         .rename("protein_id"))
    brite = (brite[brite != "None"]
         .dropna()
         .reset_index()
         .set_index("protein_id")
         .rename(columns={'index':'ko_brite'}))
    brite=brite[brite['ko_brite']!="ko01000 - Enzymes"]
    brite=brite.groupby("protein_id").agg(list)
    brite.rename(columns={'index':'ko_brite'},inplace=True)
    #delete duplicates of protein_id
    #brite=brite[~brite.index.duplicated(keep='first')]
    pathway = (df_pathway_genes.loc[name]
         .explode()
         .rename("protein_id"))
    pathway = (pathway[pathway != "None"]
            .dropna()
            .reset_index()
            .set_index("protein_id"))
    pathway=pathway.groupby("protein_id").agg(list)
    pathway.rename(columns={'index':'pathway'},inplace=True)
    
    #delete duplicates of protein_id
    #pathway=pathway[~pathway.index.duplicated(keep='first')]
    #find file in refseq with name in it
    for file2 in refseq.glob(f"*{name}*.gbff"):
        genoma = SeqIO.parse(file2, "genbank")
        for seq in genoma:
            for record in seq.features:
                if record.type == "CDS":
                    if "locus_tag" in record.qualifiers:
                        locus_tag = record.qualifiers["locus_tag"][0]
                        GO_component = record.qualifiers.get("GO_component", ["None"])
                        GO_process = record.qualifiers.get("GO_process", ["None"])
                        GO_function = record.qualifiers.get("GO_function", ["None"])
                        protein_id = record.qualifiers.get("protein_id", ["None"])[0]
                        gene=record.qualifiers.get("gene", ["None"])[0]
                        product=record.qualifiers.get("product", ["None"])[0]
                        if locus_tag in motivi.index:
                            motivi.loc[locus_tag,"location"] = str(record.location)
                            motivi.loc[locus_tag,"gene"] = gene
                            motivi.loc[locus_tag,"protein_id"] = protein_id
                            motivi.loc[locus_tag,"product"] = product
                            motivi.loc[locus_tag,"GO_component"] = GO_component[0]
                            motivi.loc[locus_tag,"GO_process"] = GO_process[0]
                            motivi.loc[locus_tag,"GO_function"] = GO_function[0]
    #join with brite

    
    motivi=motivi.join(brite,how="left", on="protein_id")
    #join with pathway
    motivi=motivi.join(pathway,how="left", on="protein_id")
    motivi['ko_brite'] = motivi['ko_brite'].apply(lambda d: d if isinstance(d, list) else [])
    flattened_kos = pd.Series([ko for sublist in motivi['ko_brite'] for ko in sublist])

    # Get the count of each KO
    ko_counts = flattened_kos.value_counts()
    
    # Create a dictionary with KOs as keys and their counts as values
    ko_dict = ko_counts.to_dict()
    
    # Create a new column 'ko_count' in the DataFrame that contains the count of the most common KO in each list
    motivi['ko_count'] = motivi['ko_brite'].apply(lambda x: 0 if x==[] else max([ko_dict.get(ko, 0) for ko in x]))
    
    # Sort the DataFrame based on 'ko_count'
    motivi = motivi.sort_values(by='ko_count', ascending=False)
    # motivi['group_size'] = motivi.groupby('ko_brite')['ko_brite'].transform('size')
    # 
    # # Sort the DataFrame by the 'group_size' column in descending order
    # motivi = motivi.sort_values(by='group_size', ascending=False)
    #motivi = motivi.drop(columns='group_size')
    # If you don't need the 'group_size' column anymore, you can drop it
    
    motivi=motivi[["protein_id","gene","product","motivo","location","pathway","ko_brite","GO_component","GO_process","GO_function"]]
    motivi.to_excel(f"/home/davide/Desktop/genomiChro/motiviannotati/{name}.xlsx",index=False)
    motivi.to_csv(f"/home/davide/Desktop/genomiChro/motiviannotati/{name}.csv",index=False)
    #edit xlsx file changing font and putting bold
motivi

,protein_id,gene,product,motivo,location,pathway,ko_brite,GO_component,GO_process,GO_function
locus,,,,,,,,,,
CHRO_RS08445,WP_015153781.1,None,TonB-dependent receptor,AAGTGCAAACGCACTT,[1905603:1907733](+),NaN,[ko02000 - Transporters],None,GO:0015891 - siderophore transport [Evidence IEA],GO:0015267 - channel activity [Evidence IEA]
CHRO_RS02135,WP_015152528.1,None,ABC transporter ATP-binding protein,TACAGCAGTTGTACTA,[478076:479024](-),NaN,[ko02000 - Transporters],None,None,GO:0005524 - ATP binding [Evidence IEA]
CHRO_RS17175,WP_015155507.1,None,PhnD/SsuA/transferrin family substrate-binding...,TATTACATTTGCTCTG,[3915126:3916086](-),[ABC transporters],[ko02000 - Transporters],None,None,None
CHRO_RS17770,WP_015155618.1,None,TonB-dependent siderophore receptor,TAGAGTATCTGTACTG,[4057425:4059939](+),NaN,[ko02000 - Transporters],GO:0019867 - outer membrane [Evidence IEA],GO:0015891 - siderophore transport [Evidence IEA],GO:0015343 - siderophore-iron transmembrane tr...
CHRO_RS02545,WP_015152609.1,None,ABC transporter ATP-binding protein,AAGTGCGATCGCACTA,[579774:580434](+),[ABC transporters],[ko02000 - Transporters],None,None,GO:0005524 - ATP binding [Evidence IEA]
...,...,...,...,...,...,...,...,...,...,...
CHRO_RS05475,WP_015153188.1,None,RNA-guided endonuclease TnpB family protein,TAGTATATTCCCACTA,[1234977:1236213](-),NaN,[],None,None,None
CHRO_RS29920,WP_015156378.1,None,serine/threonine-protein kinase,TATAGTAATTGTACTG,[4999062:5001129](+),NaN,[],None,GO:0006468 - protein phosphorylation [Evidence...,GO:0004672 - protein kinase activity [Evidence...
CHRO_RS07685,WP_015153636.1,None,DUF751 family protein,TAGAATAAACGTAATT,[1737974:1738181](-),NaN,[],None,None,None


In [44]:
brite = (df_brite_genes.loc["CCMEE_29"]
         .explode()
         .rename("protein_id"))
brite = (brite[brite != "None"]
         .dropna()
         .reset_index()
         .set_index("protein_id")
         .rename(columns={'index':'ko_brite'}))
brite

,ko_brite
protein_id,
WP_250123322.1,ko01000 - Enzymes
WP_250124543.1,ko01000 - Enzymes
WP_250126058.1,ko01000 - Enzymes
WP_250125188.1,ko01000 - Enzymes
WP_250124546.1,ko01000 - Enzymes
...,...
WP_250123070.1,ko02000 - Transporters
WP_250124410.1,ko02022 - Two-component system
WP_250124543.1,ko04147 - Exosome


In [122]:
df_brite_genes
CCMEE_29brite=df_brite_genes.loc["CCMEE_29"]
#explode column
CCMEE_29brite=CCMEE_29brite.explode()
#rename column
CCMEE_29brite=CCMEE_29brite.rename("protein_id")
#delete None values
CCMEE_29brite=CCMEE_29brite[CCMEE_29brite != "None"]
CCMEE_29brite
#now reset index
CCMEE_29brite=CCMEE_29brite.reset_index()
#set index protein_id
CCMEE_29brite=CCMEE_29brite.set_index("protein_id")
CCMEE_29brite=CCMEE_29brite[CCMEE_29brite['index']!="ko01000 - Enzymes"]
CCMEE_29brite= CCMEE_29brite.groupby("protein_id").agg(list)
CCMEE_29brite=CCMEE_29brite.rename(columns={'index':'kos'})
#brite=brite[brite['ko_brite']!="ko01000 - Enzymes"]

df=CCMEE_29brite
import pandas as pd

# Assuming df is your DataFrame and 'column_name' is the column with the lists
flattened_kos = pd.Series([ko for sublist in df['kos'] for ko in sublist])


# Get the count of each KO
ko_counts = flattened_kos.value_counts()

# Create a dictionary with KOs as keys and their counts as values
ko_dict = ko_counts.to_dict()

# Create a new column 'ko_count' in the DataFrame that contains the count of the most common KO in each list
df['ko_count'] = df['kos'].apply(lambda x: max([ko_dict.get(ko, 0) for ko in x]))

# Sort the DataFrame based on 'ko_count'
df = df.sort_values(by='ko_count', ascending=False)
df['kos']
#ow if index dupliacatei want only one row but wit merged values of column

protein_id
WP_250123666.1    [ko03400 - DNA repair and recombination proteins]
WP_250121492.1    [ko03400 - DNA repair and recombination proteins]
WP_250121493.1    [ko03400 - DNA repair and recombination proteins]
WP_250121496.1    [ko03400 - DNA repair and recombination proteins]
WP_250121656.1    [ko03400 - DNA repair and recombination proteins]
WP_250124411.1    [ko03400 - DNA repair and recombination proteins]
WP_250122036.1    [ko03400 - DNA repair and recombination proteins]
WP_250122517.1    [ko03032 - DNA replication proteins, ko03400 -...
WP_250121220.1    [ko01002 - Peptidases and inhibitors, ko03400 ...
WP_250123933.1    [ko03032 - DNA replication proteins, ko03400 -...
WP_250123153.1    [ko03032 - DNA replication proteins, ko03400 -...
WP_250123841.1                  [ko03016 - Transfer RNA biogenesis]
WP_250125134.1    [ko03016 - Transfer RNA biogenesis, ko03029 - ...
WP_250123878.1                  [ko03016 - Transfer RNA biogenesis]
WP_250124000.1                  [ko03

In [40]:
refseq_dir="/home/davide/Desktop/genomiChro/annotati_Refseq/
import os

from Bio import SeqIO

genoma = SeqIO.parse(proteineCCMEE, "genbank")
for seq in genoma:
    for record in seq.features:
        if record.type == "CDS":
            if "locus_tag" in record.qualifiers:
                locus_tag = record.qualifiers["locus_tag"][0]
                GO_component = record.qualifiers.get("GO_component", ["None"])
                GO_process = record.qualifiers.get("GO_process", ["None"])
                GO_function = record.qualifiers.get("GO_function", ["None"])
                protein_id = record.qualifiers.get("protein_id", ["None"])[0]
                gene=record.qualifiers.get("gene", ["None"])[0]
                product=record.qualifiers.get("product", ["None"])[0]
                if locus_tag in motivi.index:
                    motivi.loc[locus_tag,"location"] = str(record.location)
                    motivi.loc[locus_tag,"gene"] = gene
                    motivi.loc[locus_tag,"protein_id"] = protein_id
                    motivi.loc[locus_tag,"product"] = product
                    motivi.loc[locus_tag,"GO_component"] = GO_component[0]
                    motivi.loc[locus_tag,"GO_process"] = GO_process[0]
                    motivi.loc[locus_tag,"GO_function"] = GO_function[0]
                    
motivi.sort_values(by="gene",inplace=True,ascending=False)
motivi
#order ge

None
None
None
None
None
None
None


,motivo,location,gene,protein_id,product,GO_component,GO_process,GO_function
locus,,,,,,,,
LAU37_RS16005,TAGTGCAGATATACTA,[3252216:3253449](-),xseA,WP_250121492.1,exodeoxyribonuclease VII large subunit,GO:0009318 - exodeoxyribonuclease VII complex ...,GO:0006308 - DNA catabolic process [Evidence IEA],GO:0008855 - exodeoxyribonuclease VII activity...
LAU37_RS05120,GATTAAAAATGTTCTA,[1045043:1045694](+),upp,WP_250124546.1,uracil phosphoribosyltransferase,None,GO:0009116 - nucleoside metabolic process [Evi...,GO:0004845 - uracil phosphoribosyltransferase ...
LAU37_RS24975,CAGAGCAATCTTACTC,[5167682:5170382](+),topA,WP_250123153.1,type I DNA topoisomerase,GO:0005737 - cytoplasm [Evidence IEA],GO:0006260 - DNA replication [Evidence IEA],GO:0003677 - DNA binding [Evidence IEA]
LAU37_RS19980,TAGAATAGATGCAATT,[4115914:4116733](-),thiD,WP_250122239.1,bifunctional hydroxymethylpyrimidine kinase/ph...,None,GO:0009228 - thiamine biosynthetic process [Ev...,GO:0008902 - hydroxymethylpyrimidine kinase ac...
LAU37_RS25105,TAGAATAGCTGAACTT,[5199317:5199749](+),sepF,WP_250123172.1,cell division protein SepF,None,GO:0090529 - cell septum assembly [Evidence IEA],None
...,...,...,...,...,...,...,...,...
LAU37_RS10215,TAGTGTAGTTTTACCA,[2084973:2085318](+),None,WP_250125469.1,DUF4870 domain-containing protein,None,None,None
LAU37_RS06405,TAGTGCAATTGTACAA,[1319906:1320623](-),None,WP_250124778.1,hypothetical protein,None,None,None
LAU37_RS28940,CAGTGCAGCCATTCTA,[230448:231480](+),None,WP_250121189.1,IS630 family transposase,None,None,GO:0004803 - transposase activity [Evidence IEA]


In [51]:
df_brite_genes

,ko01000 - Enzymes,ko01001 - Protein kinases,ko01009 - Protein phosphatases and associated proteins,ko01002 - Peptidases and inhibitors,ko01003 - Glycosyltransferases,ko01005 - Lipopolysaccharide biosynthesis proteins,ko01011 - Peptidoglycan biosynthesis and degradation proteins,ko01004 - Lipid biosynthesis proteins,ko01008 - Polyketide biosynthesis proteins,ko01006 - Prenyltransferases,...,ko02022 - Two-component system,ko02035 - Bacterial motility proteins,ko04812 - Cytoskeleton proteins,ko04147 - Exosome,ko02048 - Prokaryotic defense system,ko04030 - G protein-coupled receptors,ko01504 - Antimicrobial resistance genes,ko00536 - Glycosaminoglycan binding proteins,ko00537 - Glycosylphosphatidylinositol (GPI)-anchored proteins,ko03200 - Viral proteins
SAG_2025,"[WP_317105586.1, WP_317108507.1, WP_317104870....","[WP_317106342.1, WP_317105483.1]",[],[WP_039714178.1],[],[],[],[],[WP_317106964.1],[WP_317104870.1],...,"[WP_317106342.1, WP_317105434.1, WP_317105483.1]",[WP_317106342.1],[],[],[],[],[],[],[],[]
PCC_7203,"[WP_015153605.1, WP_015156998.1, WP_219336057....",[WP_015155069.1],[],[WP_015155906.1],[],[],[WP_015154962.1],[],[],[WP_219336057.1],...,"[WP_015153506.1, WP_015155069.1]",[],[],[],[],[],[],[],[],[]
CCNUC1,"[WP_015153605.1, WP_106547394.1, WP_250015077....",[],[],"[WP_015155906.1, WP_250012819.1, WP_015153147.1]",[],[],[WP_250012243.1],[],[],[WP_250015077.1],...,[WP_015153506.1],[],[],[],[WP_250016253.1],[],[],[],[],[]
CCMEE_29,"[WP_250123322.1, WP_250124546.1, WP_250122010....",[],[],"[WP_250125712.1, WP_250121220.1]",[],[],[WP_250125712.1],[],[],[],...,[WP_250124410.1],[],[],[],[WP_256478979.1],[],[],[],[],[]
TS-821,"[WP_104544953.1, WP_104548770.1, WP_104547256....",[],[],"[WP_104548671.1, WP_246841289.1, WP_104546477.1]",[],[],[],[],[WP_104546602.1],[],...,[],[],[],[WP_210404567.1],[WP_104546599.1],[],"[WP_104548237.1, WP_104545453.1]",[],[],[]
SAG_39.79,"[WP_106168843.1, WP_241994356.1, WP_106169798....",[WP_106169547.1],[],[WP_015155906.1],[],[],[WP_106168445.1],[],[],[WP_241994356.1],...,"[WP_015153506.1, WP_106169547.1]",[],[],[],[WP_106166942.1],[],[],[],[],[]
CCALA_043,"[WP_106168843.1, WP_241994356.1, WP_106169798....",[WP_106169547.1],[],[WP_015155906.1],[],[],[WP_106168445.1],[],[],[WP_241994356.1],...,"[WP_015153506.1, WP_106169547.1]",[],[],[],[WP_106166942.1],[],[],[],[],[]
CCALA_051,"[WP_106546890.1, WP_106546803.1, WP_106547394....",[WP_233222557.1],[],[WP_015155906.1],[],[],[],[],[],[WP_233222594.1],...,"[WP_015153506.1, WP_233222557.1]",[],[],[],[WP_106546003.1],[],[],[],[],[]
[FACHB-1243],"[WP_192157179.1, WP_242038382.1, WP_015155906....",[],[],"[WP_015155906.1, WP_192161094.1, WP_192159732.1]",[],[],[],[],[WP_192160393.1],[WP_242038382.1],...,[WP_015153506.1],[],[],[],[WP_192157130.1],[],[],[],[],[]


In [52]:
names

['WP_250124683.1',
 'WP_250121527.1',
 'WP_250121243.1',
 'WP_250121242.1',
 'WP_317111452.1',
 'WP_158631908.1',
 'WP_158631908.1',
 'WP_317111453.1',
 'WP_199755850.1',
 'WP_219889224.1',
 'WP_199197564.1',
 'WP_106168793.1',
 'WP_106168793.1',
 'WP_015155077.1',
 'WP_250012394.1',
 'WP_106544983.1',
 'WP_015156012.1',
 'WP_015156012.1',
 'WP_015156012.1',
 'WP_106545218.1',
 'WP_015156012.1',
 'WP_250121656.1',
 'WP_317108223.1',
 'WP_192159955.1',
 'WP_015155906.1',
 'WP_039714178.1',
 'WP_015155906.1',
 'WP_015155906.1',
 'WP_015155906.1',
 'WP_015155906.1',
 'WP_015155906.1',
 'WP_015328680.1',
 'WP_250122141.1',
 'WP_210404530.1',
 'WP_250123933.1',
 'WP_317111526.1',
 'WP_250122412.1',
 'WP_250017677.1',
 'WP_250123932.1',
 'WP_250121220.1',
 'WP_317110959.1',
 'WP_106168112.1',
 'WP_106168112.1',
 'WP_192162050.1',
 'WP_322506169.1',
 'WP_106168124.1',
 'WP_127025067.1',
 'WP_317110961.1',
 'WP_250017676.1',
 'WP_317111525.1',
 'WP_106167282.1',
 'WP_106167282.1',
 'WP_3171058

In [231]:
file="/home/davide/Desktop/annotazioniGOkoCCMEE29.csv"
df=pd.read_csv(file)
df
#drop gene column
df.drop(columns=["gene"],inplace=True)
df
#remove duplicates
df.drop_duplicates(inplace=True)
df
#drop rows with same protein_id only first
df.drop_duplicates(subset=["protein_id"],keep="first",inplace=True)
df

,protein_id,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_brite,KEGG_pathways
0,WP_250124752.1,['GO:0006098 - pentose-phosphate shunt [Eviden...,NaN,['GO:0004345 - glucose-6-phosphate dehydrogena...,-,GO:0004345(PANTHER)|GO:0005829(PANTHER)|GO:000...,ko:K00036,K00036,GO:0006098 - pentose-phosphate shunt,ko04147 Exosome,"01100 Metabolic pathways, 01110 Biosynthesis o..."
2,WP_250123417.1,NaN,NaN,NaN,-,GO:0016491(PANTHER),ko:K00514,K00514,GO:0016117 - carotenoid biosynthetic process,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o..."
3,WP_250121918.1,NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN
4,WP_250124094.1,['GO:0042254 - ribosome biogenesis [Evidence I...,['GO:0005737 - cytoplasm [Evidence IEA]'],['GO:0003924 - GTPase activity [Evidence IEA]'...,-,-,ko:K14540,K14540,NaN,ko03009 Ribosome biogenesis,NaN
5,WP_250123747.1,NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6661,WP_250123219.1,NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN
6662,WP_250123218.1,NaN,NaN,NaN,-,GO:0003824(InterPro)|GO:0051536(InterPro),-,NaN,NaN,NaN,NaN
6663,WP_250123216.1,NaN,NaN,NaN,-,GO:0005887(PANTHER),ko:K03589,K03589,NaN,ko03036 Chromosome and associated proteins,04112 Cell cycle - Caulobacter
6664,WP_275983398.1,NaN,NaN,NaN,"GO:0005575,GO:0005576",NaN,ko:K14645,NaN,NaN,NaN,NaN


In [232]:
#set index protein_id
df.set_index("protein_id",inplace=True)
#join motivi with df
motivi=motivi.join(df,how="inner")
motivi

,gene,motivo,location,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_brite,KEGG_pathways
protein_id,,,,,,,,,,,,,
WP_250121492.1,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,ko03400 DNA repair and recombination proteins,03430 Mismatch repair
WP_250124546.1,upp,GATTAAAAATGTTCTA,[1045043:1045694](+),['GO:0009116 - nucleoside metabolic process [E...,NaN,['GO:0004845 - uracil phosphoribosyltransferas...,"GO:0003674,GO:0003824,GO:0004849,GO:0005575,GO...",GO:0004845(InterPro)|GO:0006223(InterPro),ko:K00761,K00761,GO:0006223 - uracil salvage,NaN,"01100 Metabolic pathways, 01232 Nucleotide met..."
WP_250123153.1,topA,CAGAGCAATCTTACTC,[5167682:5170382](+),['GO:0006260 - DNA replication [Evidence IEA]'...,['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...",-,-,ko:K03168,K03168,GO:0006265 - DNA topological change,"ko03032 DNA replication proteins, ko03400 DNA ...",NaN
WP_250122239.1,thiD,TAGAATAGATGCAATT,[4115914:4116733](-),['GO:0009228 - thiamine biosynthetic process [...,NaN,['GO:0008902 - hydroxymethylpyrimidine kinase ...,-,-,ko:K00941,K00941,NaN,NaN,"01100 Metabolic pathways, 01240 Biosynthesis o..."
WP_250123172.1,sepF,TAGAATAGCTGAACTT,[5199317:5199749](+),['GO:0090529 - cell septum assembly [Evidence ...,NaN,NaN,-,GO:0090529(InterPro),ko:K09772,K09772,NaN,ko03036 Chromosome and associated proteins,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250125818.1,None,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN
WP_250125469.1,None,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN
WP_250124778.1,None,TAGTGCAATTGTACAA,[1319906:1320623](-),NaN,NaN,NaN,-,-,-,NaN,NaN,NaN,NaN


In [237]:
#order by KEGG_brite by number of values
motivi= motivi.sort_values(by='KEGG_brite', ascending=False, na_position='last')
motivi
df=motivi
# Assuming 'column_name' is the name of the column you want to sort by
df['cluster_size'] = df.groupby('KEGG_brite')['KEGG_brite'].transform('size')

# Sort the DataFrame by the 'cluster_size' column in descending order
df = df.sort_values(by='cluster_size', ascending=False)



# Drop the 'cluster_size' column if it's no longer needed
df = df.drop(columns='cluster_size')
df
#put KEGG_brite as first column
df = df[["KEGG_brite"]+[col for col in df.columns if col not in ["KEGG_brite"]]]
df

,KEGG_brite,gene,motivo,location,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways
protein_id,,,,,,,,,,,,,
WP_250121496.1,ko03400 DNA repair and recombination proteins,None,CAGTAAAACTTCACTC,[3256115:3257708](-),NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN
WP_250121493.1,ko03400 DNA repair and recombination proteins,recA,TAGTATATCTGCACTA,[3253689:3254766](+),"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination
WP_250121656.1,ko03400 DNA repair and recombination proteins,None,TAGTAAATTTGTACTA,[3439950:3442179](-),NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination
WP_250123666.1,ko03400 DNA repair and recombination proteins,None,CAGTGCAAATGTACTG,[78399:81426](+),NaN,NaN,NaN,-,GO:0006302(InterPro)|GO:0016887(InterPro),ko:K03546,K03546,GO:0006302 - double-strand break repair,NaN
WP_250122036.1,ko03400 DNA repair and recombination proteins,None,TAGTACATTGTTACTA,[3874281:3875715](+),NaN,NaN,NaN,-,-,ko:K01669,K01669,GO:0006139 - nucleobase-containing compound me...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,None,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o..."
WP_250125818.1,NaN,None,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN
WP_250125469.1,NaN,None,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN


In [238]:
motivi.to_csv("/home/davide/Desktop/motivi.csv")

In [18]:
file="/home/davide/Downloads/motivi29_google.csv"
import pandas as pd
df=pd.read_csv(file)
df.set_index("protein_id",inplace=True)
df

,Unnamed: 0,gene,motivo,location,KEGG_brite,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways,cluster_size
protein_id,,,,,,,,,,,,,,,
NaN,Riparo del DNA e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WP_250121492.1,NaN,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),ko03400 DNA repair and recombination proteins,['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,03430 Mismatch repair,7.0
WP_250121496.1,NaN,NaN,CAGTAAAACTTCACTC,[3256115:3257708](-),ko03400 DNA repair and recombination proteins,NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN,7.0
WP_250121493.1,NaN,recA,TAGTATATCTGCACTA,[3253689:3254766](+),ko03400 DNA repair and recombination proteins,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination,7.0
WP_250121656.1,NaN,NaN,TAGTAAATTTGTACTA,[3439950:3442179](-),ko03400 DNA repair and recombination proteins,NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,NaN,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o...",NaN
WP_250125818.1,NaN,NaN,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN
WP_250125469.1,NaN,NaN,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN


In [19]:
file_genoma="/home/davide/Desktop/genomiChro/annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
from Bio import SeqIO


genoma = SeqIO.parse(file_genoma, "genbank")
for seq in genoma:
    for record in seq.features:
        if record.type == "CDS":
            if "protein_id" in record.qualifiers:
                protein_id = record.qualifiers["protein_id"][0]
                if protein_id in df.index and record.qualifiers["product"][0]!="hypothetical protein" :
                    df.loc[protein_id,"product"] = record.qualifiers["product"][0]


In [22]:
df

,Unnamed: 0,gene,motivo,location,KEGG_brite,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways,cluster_size,product
protein_id,,,,,,,,,,,,,,,,
NaN,Riparo del DNA e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WP_250121492.1,NaN,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),ko03400 DNA repair and recombination proteins,['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,03430 Mismatch repair,7.0,exodeoxyribonuclease VII large subunit
WP_250121496.1,NaN,NaN,CAGTAAAACTTCACTC,[3256115:3257708](-),ko03400 DNA repair and recombination proteins,NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN,7.0,ATP-dependent DNA helicase
WP_250121493.1,NaN,recA,TAGTATATCTGCACTA,[3253689:3254766](+),ko03400 DNA repair and recombination proteins,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination,7.0,recombinase RecA
WP_250121656.1,NaN,NaN,TAGTAAATTTGTACTA,[3439950:3442179](-),ko03400 DNA repair and recombination proteins,NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination,7.0,ATP-dependent RecD-like DNA helicase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,NaN,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o...",NaN,acetolactate synthase large subunit
WP_250125818.1,NaN,NaN,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN,"L,D-transpeptidase"
WP_250125469.1,NaN,NaN,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN,DUF4870 domain-containing protein


In [23]:
df.to_csv("/home/davide/Desktop/motivi29_google.csv")

In [48]:
file="/home/davide/Downloads/motivi29_google.csv"
file_genoma = "/home/davide/Desktop/genomiChro/annotati_Refseq/Chroococcidiopsis_sp._CCMEE_29_(cyanobacteria)_GCF_023558375.gbff"
intergen="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta"
import pandas as pd
import re
df=pd.read_csv(file)
df.set_index("protein_id",inplace=True)
motivi=['.AGT.{4,11}ACT.', 'TAGT.{5,11}CTA.']
def cerca_motivo(motivo,seq):
    matches = [m.group() for m in re.finditer(motivo, str(seq.seq))]
    return matches
    
    
genoma = SeqIO.parse(file_genoma, "genbank")

for seq in genoma:
    for record in seq.features:
        if record.type == "CDS":
            if "protein_id" in record.qualifiers:
                protein_id = record.qualifiers["protein_id"][0]
                locus_tag = record.qualifiers["locus_tag"][0]
                fasta=SeqIO.parse(intergen, "fasta")
                #print(locus_tag)
                if protein_id in df.index:
                    for seq in fasta:
                        if str(seq.id)==str(locus_tag):
                            trova=cerca_motivo(motivi[0],seq)
                            df.loc[protein_id,"motivo .AGT.{4,11}ACT."]=trova[0] if len(trova)>0 else "None"
                            trova=cerca_motivo(motivi[1],seq)
                            df.loc[protein_id,"motivo TAGT.{5,11}CTA."]=trova[0] if len(trova)>0 else "None"
df
        




,Unnamed: 0,gene,motivo,location,KEGG_brite,GO_process,GO_component,GO_function,GOs_eggnog,GOs_interpro,ko_eggnog,ko_blastkoala,GO_process_pannzer2,KEGG_pathways,cluster_size,"motivo .AGT.{4,11}ACT.","motivo TAGT.{5,11}CTA."
protein_id,,,,,,,,,,,,,,,,,
NaN,Riparo del DNA e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WP_250121492.1,NaN,xseA,TAGTGCAGATATACTA,[3252216:3253449](-),ko03400 DNA repair and recombination proteins,['GO:0006308 - DNA catabolic process [Evidence...,['GO:0009318 - exodeoxyribonuclease VII comple...,['GO:0008855 - exodeoxyribonuclease VII activi...,-,GO:0003676(InterPro),ko:K03601,K03601,NaN,03430 Mismatch repair,7.0,GAGTCTAGACTA,TAGTGCAGATATACTAT
WP_250121496.1,NaN,NaN,CAGTAAAACTTCACTC,[3256115:3257708](-),ko03400 DNA repair and recombination proteins,NaN,NaN,NaN,-,NaN,ko:K03722,K03722,NaN,NaN,7.0,CAGTAAAACTTCACTC,None
WP_250121493.1,NaN,recA,TAGTATATCTGCACTA,[3253689:3254766](+),ko03400 DNA repair and recombination proteins,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",GO:0003697(InterPro)|GO:0005524(InterPro)|GO:0...,ko:K03553,K03553,GO:0006310 - DNA recombination,03440 Homologous recombination,7.0,TAGTATATCTGCACTA,TAGTATATCTGCACTAG
WP_250121656.1,NaN,NaN,TAGTAAATTTGTACTA,[3439950:3442179](-),ko03400 DNA repair and recombination proteins,NaN,NaN,['GO:0004386 - helicase activity [Evidence IEA]'],-,GO:0006310(PANTHER)|GO:0009338(PANTHER)|GO:001...,ko:K03581,K03581,GO:0032508 - DNA duplex unwinding,03440 Homologous recombination,7.0,TAGTAAATTTGTACTA,TAGTAAATTTGTACTAA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_250124389.1,NaN,NaN,TAGTAAAATCTTAATC,[870964:872602](-),NaN,NaN,NaN,['GO:0000287 - magnesium ion binding [Evidence...,-,-,ko:K01652,K01652,NaN,"01100 Metabolic pathways, 01110 Biosynthesis o...",NaN,None,None
WP_250125818.1,NaN,NaN,TAGTGCAAACAATCTA,[2495981:2496932](-),NaN,NaN,NaN,NaN,-,GO:0016740(InterPro),-,NaN,NaN,NaN,NaN,None,TAGTGCAAACAATCTAG
WP_250125469.1,NaN,NaN,TAGTGTAGTTTTACCA,[2084973:2085318](+),NaN,NaN,NaN,NaN,-,-,ko:K09940,K09940,NaN,NaN,NaN,None,None


In [49]:
df.to_csv("/home/davide/Desktop/motivi29conricerca.csv")